## Construção de Pipeline RAG para IA Generativa com Databricks e Python

## 1. Configuração de Ambiente

Nesta etapa, realizamos a instalação e importação dos pacotes necessários para execução do pipeline.
O ambiente foi preparado de forma a garantir todas as dependências compatíveis para o processamento de dados e a operação dos modelos de IA.

In [ ]:
!pip install -q "numpy==1.25.2" databricks-langchain langchain_milvus langchain-huggingface sentence-transformers beautifulsoup4

In [2]:
!pip install -q watermark

In [3]:
!pip install -q databricks-cli databricks-sql-connector

In [5]:
import os
os.environ["USER_AGENT"] = "LangChain (Google Colab)"

In [6]:
# Imports
import bs4
import sentence_transformers
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from databricks_langchain import ChatDatabricks
from langchain_milvus import Milvus
import warnings
warnings.filterwarnings('ignore')

print("Ambiente Colab funcionando com sucesso.")


Ambiente Colab funcionando com sucesso.


## 2. Configuração de Credenciais de Acesso

Nesta etapa, são definidas as credenciais de autenticação necessárias para conexão segura com o ambiente Databricks, garantindo acesso autorizado aos recursos da plataforma.


In [7]:
# Defina as credenciais do Databricks
os.environ["DATABRICKS_HOST"] = "https://dbc-baba4cbd-2594.cloud.databricks.com/"   # Substitua pela URL do seu Databricks
os.environ["DATABRICKS_TOKEN"] = "DATABRICKS_TOKEN"           # Substitua pelo token gerado

## 3. Extração de Dados da Web e Criação de Chunks

Realiza-se a extração dos dados a partir da fonte web definida. Em seguida, o conteúdo bruto é segmentado em pequenos blocos (chunks) de texto, otimizando o processamento e a geração de embeddings para consultas posteriores.


In [8]:
# Cria o carregador de dados da web
loader = WebBaseLoader("https://mitsloanreview.com.br/tecnologia/")

In [9]:
# Executa o carregador e extrai os dados da web
documentos = loader.load()

In [10]:
# Cria o separador de texto
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1800, chunk_overlap = 200)

In [11]:
# Aplica o separador e cria os chunks (documentos)
docs = text_splitter.split_documents(documentos)

In [12]:
len(docs)

27

## 4. Carregamento do Modelo de Embeddings

Inicializa-se o modelo de embeddings pré-treinado, que transforma os blocos de texto em vetores numéricos. Essa etapa é fundamental para permitir buscas semânticas no banco vetorial.


https://huggingface.co/BAAI/bge-small-en-v1.5

In [13]:
# Carrega o modelo de embeddings
embeddings = HuggingFaceEmbeddings(model_name = "BAAI/bge-small-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 5. Criação e Carregamento do Banco de Dados Vetorial

Configura-se o banco de dados vetorial, responsável por armazenar e indexar os embeddings gerados. Essa estrutura viabiliza a recuperação eficiente dos dados relevantes para a geração de respostas.

In [14]:
# Cria o banco de dados vetorial
vector_db = Milvus.from_documents(documents = docs,
                                      embedding = embeddings,
                                      collection_name = 'collection',
                                      index_params = {"index_type": "FLAT"},
                                      connection_args = {"uri": "./milvus.db"})

In [15]:
# Cria o retriever para recuperar os dados do Vector DB
retriever = vector_db.as_retriever()

In [16]:
retriever

VectorStoreRetriever(tags=['Milvus', 'HuggingFaceEmbeddings'], vectorstore=<langchain_milvus.vectorstores.milvus.Milvus object at 0x794e3e56bb10>, search_kwargs={})

In [17]:
# Testando o Retriever

# Define uma frase
query = "Quais são as tendências de tecnologia discutidas no blog?"

# Busca texto similar a frase dentro do banco vetorial
vector_db.similarity_search(query, k = 1)

[Document(metadata={'description': 'Aprofunde o seu conhecimento sobre tecnologia e saiba como ela está revolucionando o mundo em diversos setores.', 'language': 'pt-BR', 'pk': 457618060044664833, 'source': 'https://mitsloanreview.com.br/tecnologia/', 'title': 'Tecnologia: da revolução aos impactos nos dias atuais'}, page_content='INOVAÇÃO 20 min de leitura \n\n\t\t\tTecnologia: da revolução aos impactos nos dias atuais\t\t\nSaiba como as inovações estão moldando um novo mundo –e como se adaptar a ele \n\n\n \n\n\n\t\t\t\t\t\tRedação MIT Sloan Review Brasil\t\t\t\t\t\n\n19 de julho de 2024\nAtualizado em 22 de janeiro de 2025\n\n\n\nSalvar conteúdo 2 \n\n\n\n\n\n\n\nEste conteúdo pertence à editoria Tecnologia, IA e dados Ver mais conteúdos\nLeia a seguir\n\n\n\n\n\n\n\n \n                por \n                \n                    Albert H. Segars                \n\n\nAs 7 tecnologias que estão reinventando o mundo\n\n\n\n\n\n\n\n\n\n\n por \n                \n                    Maxi

## 6. Definindo o Endpoint do Modelo de Linguagem (LLM)

Nesta etapa, configuramos o endpoint do LLM hospedado no Databricks, que será utilizado para a geração das respostas a partir dos dados recuperados do banco vetorial.

In [18]:
# Definimos aqui o endpoint para o LLM no Databricks
llm = ChatDatabricks(endpoint = "databricks-dbrx-instruct", max_tokens = 200)

## 7. Definindo o Prompt Template

Configuramos o template de prompt que orienta o modelo de linguagem na construção das respostas, garantindo consistência e direcionamento conforme o contexto dos dados extraídos.

In [19]:
# Cria o texto do prompt
PROMPT_TEMPLATE = """
        Humano: Você é um assistente de IA e fornece respostas a perguntas do usuário.

        Use as seguintes informações para fornecer uma resposta concisa à pergunta entre as tags <question>.
        Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta.

        <context>
        {context}
        </context>

        <question>
        {question}
        </question>

        A resposta deve ser específica e usar apenas informações confiáveis.

        Assistente:"""

In [20]:
# Cria o prompt template
prompt = PromptTemplate(template = PROMPT_TEMPLATE, input_variables = ["context", "question"])

In [21]:
# Cria o retriever
retriever = vector_db.as_retriever()

In [22]:
# Função para formatar os dados
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## 8. Definindo o RAG Chain para o Pipeline

Montamos a cadeia (chain) de componentes responsável por integrar a recuperação dos dados relevantes (Retrieval) e a geração de respostas (Generation) no pipeline RAG.

In [23]:
# RAG Chain

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

O `RunnablePassthrough` no LangChain é um componente que simplesmente encaminha os dados de entrada para a saída sem realizar alterações. Funciona como um "canal direto" de passagem. Neste projeto, utilizamos o `RunnablePassthrough` para transmitir a questão do usuário sem modificações até a próxima etapa do pipeline.

## 9. Executando o Pipeline

Com todos os componentes configurados, a execução do pipeline consiste em enviar uma pergunta, processá-la por meio da arquitetura RAG e retornar a resposta gerada com base nos documentos carregados.

In [24]:
# Invoca a chain
resposta = rag_chain.invoke(query)

In [25]:
resposta

'As tendências de tecnologia discutidas no blog incluem a influência da tecnologia na comunicação e conectividade, com o advento da internet e smartphones, redes sociais, videoconferências, mensagens e sistemas de gestão de projetos. Além disso, a tecnologia também tem impactado o desenvolvimento econômico e a ciência, com a ciência de dados e a inteligência artificial se destacando. Outros eventos importantes na área de tecnologia são as conferências e eventos, como a CES, a Web Summit e a Mobile World Congress.'

In [32]:
# Caixa de digitação para fazer perguntas
print("\n🔎 Pergunte algo sobre o conteúdo carregado:")
question = input("> Sua pergunta: ")

print("\n⏳ Gerando resposta...")
resposta = rag_chain.invoke(question)

print("\n📢 Resposta:\n")
for linha in resposta.split('\n'):
    print(linha)



🔎 Pergunte algo sobre o conteúdo carregado:
> Sua pergunta: Como a inteligência artificial está transformando os negócios?

⏳ Gerando resposta...

📢 Resposta:

A inteligência artificial (IA) está transformando os negócios de várias maneiras. Ela está impulsionando a eficiência, a inovação e a competitividade em diversos setores, permitindo que as organizações analisem grandes volumes de dados de maneira rápida e precisa para identificar padrões complexos e tomar decisões informadas em tempo real. A IA está automatizando tarefas repetitivas, reduzindo custos operacionais e aumentando a produtividade dos funcionários. Além disso, a IA está impulsionando novos modelos de negócios e criando oportunidades para startups e empreendedores inovarem com produtos e serviços disruptivos.


# Fim